## API 구분


- [네이버](https://developers.naver.com/docs/common/openapiguide/)
    - 통합 검색어 트렌드 API
    - 쇼핑인사이트 API

- [클라우드 API](https://api.ncloud-docs.com/docs/api-overview)
    - Geocoding
    - Reverse Geocoding
    - Static Map

In [1]:
from config import config

# 네이버 파파고 API
papago_id = config['naver']['papago']['client_id']
papago_pw = config['naver']['papago']['client_id']

# 네이버 통합 검색어 API
search_id = config['naver']['search']['client_id']
search_pw = config['naver']['search']['client_secret']

# 네이버 단축 URL API
shorturl_id = config['naver']['shorturl']['client_id']
shorturl_pw = config['naver']['shorturl']['client_secret']

# 네이버 쇼핑인사이트 API
shopping_id = config['naver']['shopping']['client_id']
shopping_pw = config['naver']['shopping']['client_secret']

# 네이버 클라우드 플랫폼 API
cloud_id = config['cloud']['client_id']
cloud_pw = config['cloud']['client_secret']

In [2]:
import json
import requests
import pandas as pd

## NAVER

In [22]:
class Naver:
    """
    네이버 API 클래스
    """
    def __init__(self, client_id, client_secret):
        self.headers = {
            "X-Naver-Client-Id": client_id,
            "X-Naver-Client-Secret": client_secret,
            "Content-Type": "application/json",
        }
    


    def datalab_search(self, **kwargs):
        """
        네이버 통합 검색어 트렌드 조회
        
        - startDate (STRING) 필수: 조회 기간 시작 날짜(yyyy-mm-dd 형식). 2016년 1월 1일부터 조회할 수 있습니다.
        - endDate (STRING) 필수: 조회 기간 종료 날짜(yyyy-mm-dd 형식)
        - timeUnit (STRING) 필수: 구간 단위
            - date: 일간
            - week: 주간
            - month: 월간
        - keywordGroups (LIST of DICT) 필수: 주제어와 주제어에 해당하는 검색어 묶음 쌍의 배열. 최대 5개의 쌍을 배열로 설정할 수 있습니다.
            - groupName (STRING) 필수: 주제어. 검색어 묶음을 대표하는 이름입니다.
            - keywords (LIST) 필수: 주제어에 해당하는 검색어. 최대 20개의 검색어를 배열로 설정할 수 있습니다.
        - device (STRING): 범위. 검색 환경에 따른 조건입니다.
            - 설정 안 함: 모든 환경
            - pc: PC에서 검색 추이
            - mo: 모바일에서 검색 추이
        - gender (STRING): 성별. 검색 사용자의 성별에 따른 조건입니다.
            - 설정 안 함: 모든 성별
            - m: 남성
            - f: 여성
        - ages (LIST): 연령. 검색 사용자의 연령에 따른 조건입니다.
            - 설정 안 함: 모든 연령
            - 1: 0∼12세
            - 2: 13∼18세
            - 3: 19∼24세
            - 4: 25∼29세
            - 5: 30∼34세
            - 6: 35∼39세
            - 7: 40∼44세
            - 8: 45∼49세
            - 9: 50∼54세
            - 10: 55∼59세
            - 11: 60세 이상
        """
        url = "https://openapi.naver.com/v1/datalab/search"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        
        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title','period','ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title', values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot
        
        else:
            return res.status_code


    def datalab_shopping_categories(self, **kwargs):
        """
        쇼핑인사이트 분야별 트렌드 조회
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/categories"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            
            
            return
            
        else:
            return res.status_code

    def datalab_shopping_category_device(self, **kwargs):
        """
        쇼핑인사이트 분야 내 기기별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_gender(self, **kwargs):
        """
        쇼핑인사이트 분야 내 성별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_age(self, **kwargs):
        """
        쇼핑인사이트 분야 내 연령별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_keywords(self, **kwargs):
        """
        쇼핑인사이트 키워드별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_keyword_device(self, **kwargs):
        """
        쇼핑인사이트 키워드 기기별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_keyword_gender(self, **kwargs):
        """
        쇼핑인사이트 키워드 성별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v
            
    def datalab_shopping_category_keyword_age(self, **kwargs):
        """
        쇼핑인사이트 키워드 연령별 트렌드 조회
        """
        url = ""
        data = {}
        for k, v in kwargs.items():
            data[k] = v


    def util_shorturl(self, url):
        """
        단축 URL 요청(JSON)
        """
        req_url = f"https://openapi.naver.com/v1/util/shorturl?url={url}"
        res = requests.get(req_url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### 통합 검색어 트렌드 API

In [23]:
# 네이버 API 인스턴스
naver = Naver(search_id, search_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
keywordGroups = [
    {
        "groupName": "미국",
        "keywords": ["연준","FRB","FED","FOMC"]
    },
    
    {
        "groupName": "한국",
        "keywords": ["한국은행","금융통화위원회"]
    }
]


# 요청
df = naver.datalab_search(startDate=startDate, 
                          endDate=endDate, 
                          timeUnit=timeUnit, 
                          keywordGroups=keywordGroups)
df

,날짜,미국,한국
0,2022-01-01,0.98301,0.59953
1,2022-01-02,1.19120,0.80558
2,2022-01-03,1.97032,1.79861
3,2022-01-04,2.11269,1.82866
4,2022-01-05,5.29640,1.88733
...,...,...,...
237,2022-08-26,5.09107,2.20284
238,2022-08-27,4.42643,0.96441
239,2022-08-28,2.43321,0.89072
240,2022-08-29,3.78110,2.29155


### 쇼핑인사이트 API

In [ ]:
# 네이버 API 인스턴스
naver = Naver(shopping_id, shopping_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
keywordGroups = []


# 요청
df = naver.datalab_

### 단축 URL API

In [27]:
# 네이버 API 인스턴스
naver = Naver(shorturl_id, shorturl_pw)

# 파라미터
url = "https://wooiljeong.github.io"

# 단축 URL API 호출
res = naver.util_shorturl(url)
res

{'result': {'url': 'https://me2.do/FniJlxxK',
  'hash': 'FniJlxxK',
  'orgUrl': 'https://wooiljeong.github.io'},
 'message': 'ok',
 'code': '200'}

## NAVER CLOUD PLATFORM

In [28]:
class NaverCloudPlatform:
    """
    네이버 클라우드 플랫폼 API 클래스
    """
    def __init__(self, client_id, client_secret):
        self.headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_secret,
        }
        
    def geocoding(self, query):
        """
        지오코딩 API
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={query}"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def reverse_geocoding(self, lon, lat):
        """
        리버스 지오코딩 API
        """
        coords = f"{lon},{lat}"
        url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json&orders=addr"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res
        
    def directions5(self, **kwargs):
        """
        Directions5 API
        
        - start: 출발지 (필수)
        - goal: 목적지 (필수)
            하나 이상의 목적지 정보를 전달할 수 있으며, 복수 개의 목적지를 입력할 때는 :로 연결
            입력한 목적지 정보 중 가장 적은 비용으로 도달할 수 있는 목적지로 경로가 생성됨
            목적지의 최대 개수는 10개이며, 최초의 최적 목적지 좌표를 기준으로 직선거리 3km 이내의 좌표만 유효하게 판정
        - waypoints: 경유지
            최대 5개를 입력할 수 있으며 서로 다른 경유지의 구분자로 |(pipe char)를 사용
        - option: 탐색 옵션
            옵션은 최대 3개까지 동시에 요청할 수 있으며, 여러 옵션은 ‘:'로 연결
            기본값은 traoptimal
        - cartype: 톨게이트 요금 계산용 차종 정보
            기본값은 1
            차종을 바탕으로 차량의 중량 또는 크기를 판단하지 않음
            3, 4, 5종은 option에서 traavoidtoll이 적용되지 않았더라도 하이패스 전용 톨게이트를 회피해서 탐색
            일반적인 승용차는 모두 1종
        - fueltype: 유류비 계산용 유종
            기본값은 gasoline
        - mileage: 설정된 유류에 해당하는 연비
            사용하는 차량의 연비를 사용자가 직접 입력하고자 할 때 사용하는 파라미터
            기본값은 14
        - lang: 언어 종류
            기본값은 ko
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def directions15(self, **kwargs):
        """
        Directions15 API
        
        - start: 출발지 (필수)
        - goal: 목적지 (필수)
            하나 이상의 목적지 정보를 전달할 수 있으며, 복수 개의 목적지를 입력할 때는 :로 연결
            입력한 목적지 정보 중 가장 적은 비용으로 도달할 수 있는 목적지로 경로가 생성됨
            목적지의 최대 개수는 10개이며, 최초의 최적 목적지 좌표를 기준으로 직선거리 3km 이내의 좌표만 유효하게 판정
        - waypoints: 경유지
            최대 15개를 입력할 수 있음
            서로 다른 경유지의 구분자로 |(pipe char)를 사용
        - option: 탐색 옵션
            옵션은 최대 3개까지 동시에 요청할 수 있음
            여러 옵션은 ‘:'로 연결하며 기본값은 traoptimal
        - cartype: 톨게이트 요금 계산용 차종 정보
            기본값은 1
            차종을 바탕으로 차량의 중량 또는 크기를 판단하지는 않음
            3, 4, 5종은 option에서 traavoidtoll이 적용되지 않았더라도 하이패스 전용 톨게이트를 회피해서 탐색
            일반적인 승용차는 모두 1종
        - fueltype: 유류비 계산용 유종
            기본값은 gasoline
        - mileage: 설정된 유류에 해당하는 연비
            사용하는 차량의 연비를 사용자가 직접 입력하고자 할 때 사용하는 파라미터
            기본값은 14
        - lang: 언어 종류
            기본값은 ko
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### Directions5

In [18]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions5(start=start, goal=goal)
res

{'code': 0,
 'message': '길찾기를 성공하였습니다.',
 'currentDateTime': '2022-08-31T15:11:36',
 'route': {'traoptimal': [{'summary': {'start': {'location': [127.111505,
       37.3864536]},
     'goal': {'location': [127.0195345, 37.51608], 'dir': 2},
     'distance': 19972,
     'duration': 2397820,
     'etaServiceType': 1,
     'departureTime': '2022-08-31T15:11:36',
     'bbox': [[127.0137745, 37.3863436], [127.1130969, 37.5228656]],
     'tollFare': 1000,
     'taxiFare': 20990,
     'fuelPrice': 2484},
    'path': [[127.1115011, 37.3863436],
     [127.1113757, 37.3863459],
     [127.1113214, 37.3863457],
     [127.1113223, 37.3864025],
     [127.1113213, 37.3865918],
     [127.1113229, 37.3869451],
     [127.1113227, 37.3869785],
     [127.1113209, 37.3873291],
     [127.1113207, 37.3873625],
     [127.1113202, 37.3874562],
     [127.1113186, 37.3877735],
     [127.1113184, 37.3878141],
     [127.1113193, 37.3878519],
     [127.1113189, 37.387933],
     [127.1114544, 37.3879344],
     [127.

### Directions15

In [20]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions15(start=start, goal=goal)
res

{'code': 0,
 'message': '길찾기를 성공하였습니다.',
 'currentDateTime': '2022-08-31T15:15:23',
 'route': {'traoptimal': [{'summary': {'start': {'location': [127.111505,
       37.3864536]},
     'goal': {'location': [127.0195345, 37.51608], 'dir': 2},
     'distance': 24744,
     'duration': 2955269,
     'etaServiceType': 1,
     'departureTime': '2022-08-31T15:15:23',
     'bbox': [[127.0137745, 37.3819378], [127.1163827, 37.5228656]],
     'tollFare': 0,
     'taxiFare': 26250,
     'fuelPrice': 3078},
    'path': [[127.1115011, 37.3863436],
     [127.1115847, 37.3863421],
     [127.1115977, 37.3860023],
     [127.1115998, 37.385822],
     [127.1115998, 37.3858139],
     [127.1115975, 37.3856147],
     [127.1116059, 37.3853002],
     [127.1116073, 37.385238],
     [127.1121811, 37.3852426],
     [127.1124398, 37.3852515],
     [127.1126294, 37.385281],
     [127.1126983, 37.3852758],
     [127.1127221, 37.3852687],
     [127.1127741, 37.3852544],
     [127.1128274, 37.3852221],
     [127.11284

### Geocoding

In [6]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 주소
query = "백현동 560"

# 지오코딩 요청
res = ncp.geocoding(query)
res

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '경기도 성남시 분당구 판교역로2번길 40',
   'jibunAddress': '경기도 성남시 분당구 백현동 560',
   'englishAddress': '40, Pangyoyeok-ro 2beon-gil, Bundang-gu, Seongnam-si, Gyeonggi-do, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '경기도',
     'shortName': '경기도',
     'code': ''},
    {'types': ['SIGUGUN'],
     'longName': '성남시 분당구',
     'shortName': '성남시 분당구',
     'code': ''},
    {'types': ['DONGMYUN'], 'longName': '백현동', 'shortName': '백현동', 'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '판교역로2번길',
     'shortName': '판교역로2번길',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '40',
     'shortName': '40',
     'code': ''},
    {'types': ['BUILDING_NAME'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '560',
     'shortName': '560',
 

### Reverse Geocoding

In [21]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 경도, 위도
lon, lat = 127.1115061, 37.3864539

# 지오코딩 요청
res = ncp.reverse_geocoding(lon, lat)
res

{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'addr',
   'code': {'id': '4113511000', 'type': 'L', 'mappingId': '02135110'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '경기도',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.550802,
       'y': 37.4363177}},
     'alias': '경기'},
    'area2': {'name': '성남시 분당구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.1189255,
       'y': 37.3828195}}},
    'area3': {'name': '백현동',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 127.1108299,
       'y': 37.3885485}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}},
   'land': {'type': '1',
    'number1': '560',
    'number2': '',
    'addition0': {'type': '', 'value': ''},
    'addition1': {'type': '', 'value': ''},
    'addition2': {'type': '', 'value': ''},
    'addition3': {'type': '', 'value': ''},
    'a